In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras
import sys
import PIL
import os
import tensorflow.keras.backend as K
import glob
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
import segmentation_models as sm
sm.set_framework('tf.keras')
import multiprocessing as mp
from save_weights_every_epoch import CallbackForSavingModelWeights
from change_learning_rate_epoch import ChangeLR
import multiprocessing as mp

Segmentation Models: using `keras` framework.


In [2]:
load_dotenv('../config_files/dev.env')

True

In [3]:
csv_file = pd.read_csv('../../files/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()

In [4]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../files/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../files/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [5]:
for x in tqdm(csv_file['fixed_inputs'].values.tolist()):
    if os.path.exists(x) == False:
        print(x)

100%|██████████| 192556/192556 [00:00<00:00, 298750.79it/s]


In [6]:
for x in tqdm(csv_file['mask_paths'].values.tolist()):
    if os.path.exists(x) == False:
        print(x)

100%|██████████| 192556/192556 [00:00<00:00, 292476.16it/s]


In [7]:
csv_file['fixed_inputs'].values.tolist()[0]

'../../files/train_v2/00003e153.jpg'

In [8]:
csv_file = csv_file[csv_file['fixed_inputs'] != '../../files/train_v2/6384c3e78.jpg']

In [9]:
allowed_gpus = [0, 1, 2, 3]
gpus = tf.config.list_physical_devices("GPU")
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, "GPU")
for gpu in final_gpu_list:
    tf.config.experimental.set_memory_growth(gpu, True)

strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')


In [10]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size = test_size, random_state=42)
    train, val = train_test_split(train, test_size = test_size, random_state=42)
    return train, val, test

In [11]:
# csv_file = pd.read_csv
train, val, test = split_datasets(csv_file)

In [12]:
def get_model(model_name=None):
    with strategy.scope():
        if model_name == None:
            model = sm.Linknet()
        else:
            model = sm.Linknet(model_name)
        model.compile(
            tf.keras.optimizers.SGD(),
            loss = sm.losses.bce_dice_loss,
            metrics = [sm.metrics.iou_score]
        )
    return model

In [13]:
def read_imgs(img, mask, shape):
    img = tf.io.read_file(img)
    mask = tf.io.read_file(mask)
    img = tf.image.decode_jpeg(img, channels=3)
    mask = tf.image.decode_jpeg(mask, channels=1)
    img = tf.image.resize(img, size=shape)
    mask = tf.image.resize(mask, size=shape)
    img = img / 255
    mask = mask / 255
    return img, mask

In [14]:
def get_data(data, test_ = False, aug_seq = None, shape = (256, 256), shuffle = True, repeat = True, batch = True, batch_size = 32):
    imgs, masks = data['fixed_inputs'].values.tolist(), data['mask_paths'].values.tolist()
    shapes = [shape for x in range(len(imgs))]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, masks, shapes))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(256 * REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    tensor = tensor.map(read_imgs)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [15]:
os.environ['tb_path']

'/home/ubuntu/ship_segmentation/TB/'

In [34]:
K.clear_session()
batch_size = 256
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/res50_baseline_{batch_size}_is_32/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/res50_baseline_{batch_size}_is_32/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (32, 32), batch_size=batch_size)
val_dataset = get_data(val, shape = (32, 32), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model()
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
92/92 [==============================] - 43s 148ms/step - loss: 1.1817 - iou_score: 0.0010 - val_loss: 1.0361 - val_iou_score: 8.1844e-04
Epoch 2/50
92/92 [==============================] - 9s 100ms/step - loss: 1.0422 - iou_score: 0.0013 - val_loss: 1.0332 - val_iou_score: 0.0013
Epoch 3/50
92/92 [==============================] - 9s 100ms/step - loss: 1.0236 - iou_score: 0.0017 - val_loss: 1.0202 - val_iou_score: 0.0020
Epoch 4/50
92/92 [==============================] - 8s 90ms/step - loss: 1.0159 - iou_score: 0.0021 - val_loss: 1.0134 - val_iou_score: 0.0026
Epoch 5/50
92/92 [==============================] - 8s 90ms/step - loss: 1.0111 - iou_score: 0.0026 - val_loss: 1.0089 - val_iou_score: 0.0034
Epoch 6/50
92/92 [==============================] - 10s 107ms/step - loss: 1.0074 - iou_score: 0.0033 - val_loss: 1.0052 - val_iou_score: 0.0043
Epoch 7/50
92/92 [==============================] - 7s 78ms/step - loss: 1.0039 - iou_score: 0.0043 - val_loss: 1.0013 - val_iou_sco

In [35]:
K.clear_session()
batch_size = 256
image_size = 64
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/psp_baseline_{batch_size}_is_{image_size}/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/psp_baseline_{batch_size}_is_{image_size}/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (image_size, image_size), batch_size=batch_size)
val_dataset = get_data(val, shape = (image_size, image_size), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model()
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
92/92 [==============================] - 44s 146ms/step - loss: 1.1843 - iou_score: 6.7907e-04 - val_loss: 1.0718 - val_iou_score: 2.0553e-04
Epoch 2/50
92/92 [==============================] - 10s 104ms/step - loss: 1.0465 - iou_score: 8.3722e-04 - val_loss: 1.0455 - val_iou_score: 7.6960e-04
Epoch 3/50
92/92 [==============================] - 10s 106ms/step - loss: 1.0263 - iou_score: 0.0013 - val_loss: 1.0258 - val_iou_score: 0.0017
Epoch 4/50
92/92 [==============================] - 9s 102ms/step - loss: 1.0164 - iou_score: 0.0024 - val_loss: 1.0129 - val_iou_score: 0.0041
Epoch 5/50
92/92 [==============================] - 9s 103ms/step - loss: 1.0004 - iou_score: 0.0085 - val_loss: 0.9834 - val_iou_score: 0.0181
Epoch 6/50
92/92 [==============================] - 9s 103ms/step - loss: 0.9389 - iou_score: 0.0391 - val_loss: 0.8870 - val_iou_score: 0.0665
Epoch 7/50
92/92 [==============================] - 10s 105ms/step - loss: 0.7799 - iou_score: 0.1302 - val_loss: 0.6

In [36]:
K.clear_session()
batch_size = 256
image_size = 128
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/psp_baseline_{batch_size}_is_{image_size}/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/psp_baseline_{batch_size}_is_{image_size}/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (image_size, image_size), batch_size=batch_size)
val_dataset = get_data(val, shape = (image_size, image_size), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model()
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
92/92 [==============================] - 63s 323ms/step - loss: 1.2884 - iou_score: 8.4340e-04 - val_loss: 1.0839 - val_iou_score: 5.3566e-04
Epoch 2/50
92/92 [==============================] - 25s 272ms/step - loss: 1.0596 - iou_score: 6.8273e-04 - val_loss: 1.0504 - val_iou_score: 8.5325e-04
Epoch 3/50
92/92 [==============================] - 25s 272ms/step - loss: 1.0318 - iou_score: 9.5626e-04 - val_loss: 1.0298 - val_iou_score: 0.0013
Epoch 4/50
92/92 [==============================] - 25s 273ms/step - loss: 1.0211 - iou_score: 0.0014 - val_loss: 1.0199 - val_iou_score: 0.0019
Epoch 5/50
92/92 [==============================] - 25s 271ms/step - loss: 1.0147 - iou_score: 0.0021 - val_loss: 1.0133 - val_iou_score: 0.0029
Epoch 6/50
92/92 [==============================] - 25s 272ms/step - loss: 1.0091 - iou_score: 0.0034 - val_loss: 1.0065 - val_iou_score: 0.0048
Epoch 7/50
92/92 [==============================] - 25s 271ms/step - loss: 1.0011 - iou_score: 0.0062 - val_lo

In [37]:
K.clear_session()
batch_size = 256
image_size = 32
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/psp_res50_baseline_{batch_size}_is_{image_size}/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/psp_res50_baseline_{batch_size}_is_{image_size}/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (image_size, image_size), batch_size=batch_size)
val_dataset = get_data(val, shape = (image_size, image_size), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('resnet50')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

94592056/94592056 [==============================] - 4s 0us/step
Epoch 1/50
92/92 [==============================] - 125s 350ms/step - loss: 1.1991 - iou_score: 0.0010 - val_loss: 1.2434 - val_iou_score: 0.0013
Epoch 2/50
92/92 [==============================] - 19s 209ms/step - loss: 1.0540 - iou_score: 0.0010 - val_loss: 1.0933 - val_iou_score: 0.0012
Epoch 3/50
92/92 [==============================] - 19s 208ms/step - loss: 1.0320 - iou_score: 9.9825e-04 - val_loss: 1.0490 - val_iou_score: 0.0012
Epoch 4/50
92/92 [==============================] - 19s 212ms/step - loss: 1.0234 - iou_score: 9.7793e-04 - val_loss: 1.0251 - val_iou_score: 0.0011
Epoch 5/50
92/92 [==============================] - 19s 209ms/step - loss: 1.0189 - iou_score: 9.7911e-04 - val_loss: 1.0194 - val_iou_score: 0.0011
Epoch 6/50
92/92 [==============================] - 19s 210ms/step - loss: 1.0163 - iou_score: 9.6540e-04 - val_loss: 1.0189 - val_iou_score: 0.0010
Epoch 7/50
92/92 [==============================

In [38]:
K.clear_session()
batch_size = 256
image_size = 64
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/psp_res50_baseline_{batch_size}_is_{image_size}/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/psp_res50_baseline_{batch_size}_is_{image_size}/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (image_size, image_size), batch_size=batch_size)
val_dataset = get_data(val, shape = (image_size, image_size), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('resnet50')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
92/92 [==============================] - 108s 235ms/step - loss: 1.2085 - iou_score: 0.0010 - val_loss: 1.2450 - val_iou_score: 0.0013
Epoch 2/50
92/92 [==============================] - 12s 125ms/step - loss: 1.0509 - iou_score: 9.9133e-04 - val_loss: 1.0859 - val_iou_score: 0.0013
Epoch 3/50
92/92 [==============================] - 11s 125ms/step - loss: 1.0301 - iou_score: 9.8593e-04 - val_loss: 1.0386 - val_iou_score: 0.0015
Epoch 4/50
92/92 [==============================] - 11s 123ms/step - loss: 1.0222 - iou_score: 9.7449e-04 - val_loss: 1.0269 - val_iou_score: 0.0012
Epoch 5/50
92/92 [==============================] - 12s 127ms/step - loss: 1.0181 - iou_score: 9.6582e-04 - val_loss: 1.0212 - val_iou_score: 0.0011
Epoch 6/50
92/92 [==============================] - 11s 123ms/step - loss: 1.0156 - iou_score: 9.6519e-04 - val_loss: 1.0169 - val_iou_score: 0.0012
Epoch 7/50
92/92 [==============================] - 11s 125ms/step - loss: 1.0139 - iou_score: 9.6089e-04 - v

In [39]:
K.clear_session()
batch_size = 256
image_size = 128
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/psp_res50_baseline_{batch_size}_is_{image_size}/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/psp_res50_baseline_{batch_size}_is_{image_size}/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (image_size, image_size), batch_size=batch_size)
val_dataset = get_data(val, shape = (image_size, image_size), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('resnet50')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
92/92 [==============================] - 129s 391ms/step - loss: 1.2058 - iou_score: 0.0011 - val_loss: 1.1679 - val_iou_score: 0.0013
Epoch 2/50
92/92 [==============================] - 25s 268ms/step - loss: 1.0464 - iou_score: 9.5383e-04 - val_loss: 1.0664 - val_iou_score: 0.0013
Epoch 3/50
92/92 [==============================] - 25s 269ms/step - loss: 1.0277 - iou_score: 9.4996e-04 - val_loss: 1.0354 - val_iou_score: 0.0012
Epoch 4/50
92/92 [==============================] - 25s 267ms/step - loss: 1.0206 - iou_score: 9.4852e-04 - val_loss: 1.0244 - val_iou_score: 0.0012
Epoch 5/50
92/92 [==============================] - 25s 269ms/step - loss: 1.0169 - iou_score: 9.4498e-04 - val_loss: 1.0194 - val_iou_score: 0.0011
Epoch 6/50
92/92 [==============================] - 25s 269ms/step - loss: 1.0146 - iou_score: 9.3555e-04 - val_loss: 1.0166 - val_iou_score: 9.9334e-04
Epoch 7/50
92/92 [==============================] - 25s 268ms/step - loss: 1.0131 - iou_score: 9.3344e-04

In [17]:
K.clear_session()
batch_size = 128
image_size = 256
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/psp_res50_baseline_{batch_size}_is_{image_size}/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/psp_res50_baseline_{batch_size}_is_{image_size}/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (image_size, image_size), batch_size=batch_size)
val_dataset = get_data(val, shape = (image_size, image_size), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('resnet50')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
INFO:tensorflow:batch_all_reduce: 201 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 201 all-reduces with algorithm = nccl, num_packs = 1
368/368 [==============================] - 349s 807ms/step - loss: 1.0727 - iou_score: 0.0011 - val_loss: 1.0250 - val_iou_score: 0.0012
Epoch 2/50
368/368 [==============================] - 292s 795ms/step - loss: 1.0138 - iou_score: 0.0013 - val_loss: 1.0134 - val_iou_score: 0.0012
Epoch 3/50
368/368 [==============================] - 292s 794ms/step - loss: 1.0057 - iou_score: 0.0031 - val_loss: 0.9897 - val_iou_score: 0.0113
Epoch 4/50
368/368 [==============================] - 292s 794ms/step - loss: 0.8341 - iou_score: 0.1020 - val_loss: 0.5746 - val_iou_score: 0.2827
Epoch 5/50
368/368 [==============================] - 292s 793ms/step - loss: 0.3651 - iou_score: 0.4758 - val_loss: 0.3394 - val_iou_score: 0.5090
Epoch 6/50
368/368 [==============================] - 292s 793ms/step - loss: 0.2729 -

In [18]:
K.clear_session()
batch_size = 32
image_size = 512
log_dir = f"{os.environ['tb_path']}segmentation/pspnet/psp_res50_baseline_{batch_size}_is_{image_size}/"
if os.path.exists(log_dir) == False:
    os.makedirs(log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
weights_path = f'../../ship_seg_weights/segmentation/pspnet/psp_res50_baseline_{batch_size}_is_{image_size}/'
weights_save = CallbackForSavingModelWeights(weights_path)
train_dataset = get_data(train, shape = (image_size, image_size), batch_size=batch_size)
val_dataset = get_data(val, shape = (image_size, image_size), batch_size=batch_size, shuffle=False, repeat=False)
with strategy.scope():
    model = get_model('resnet50')
    model_hist = model.fit(
        train_dataset,
        steps_per_epoch = len(train) // (batch_size * REPLICAS),
        epochs = 50,
        verbose = 1,
        validation_data = val_dataset,
        callbacks = [tensorboard_callback]
    )

Epoch 1/50
INFO:tensorflow:batch_all_reduce: 201 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:batch_all_reduce: 201 all-reduces with algorithm = nccl, num_packs = 1
1474/1474 [==============================] - 1211s 788ms/step - loss: 0.9466 - iou_score: 0.0560 - val_loss: 0.4562 - val_iou_score: 0.3937
Epoch 2/50
1474/1474 [==============================] - 1158s 786ms/step - loss: 0.2638 - iou_score: 0.6031 - val_loss: 0.2387 - val_iou_score: 0.6349
Epoch 3/50
1474/1474 [==============================] - 1158s 786ms/step - loss: 0.2034 - iou_score: 0.6798 - val_loss: 0.2001 - val_iou_score: 0.6860
Epoch 4/50
1474/1474 [==============================] - 1159s 786ms/step - loss: 0.1809 - iou_score: 0.7092 - val_loss: 0.1751 - val_iou_score: 0.7140
Epoch 5/50
1474/1474 [==============================] - 1158s 786ms/step - loss: 0.1639 - iou_score: 0.7315 - val_loss: 0.1638 - val_iou_score: 0.7299
Epoch 6/50
1474/1474 [==============================] - 1157s 785ms/ste

KeyboardInterrupt: 